In [1]:
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#define utilities functions
def downcast_dtypes(df):
    """
    Downsize float64/int64 with float32/int32 to save memory usage
    """
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype in ["int64"]]

    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)

    return df

def one_hot_encoder(df, nan_as_category = True):
    """
    One-hot encoding for categorical columns with get_dummies
    """
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns


def remove_date_outlier(df):
    """
    Replace <-40000 with np.nan in date columns of df
    
    """
    for col in df.columns:
        if "DAYS" in col:
            df[col].loc[df[col] < -40000] = np.nan

    return df

def process_bureau_balancing_with_period(df,last_month_nums):
    '''
    Parameter:
    df: bureau balancing dataframe
    last_month_nums: last number of months
    
    Output:
    processed bureau balancing dataframe for specific period (month_nums)
    '''
    crit = df['MONTHS_BALANCE']>-last_month_nums
    df=df[crit]
    curr_column_header_list = [col for col in df.columns if 'STATUS' in col]
    new_column_header_list = [col+'_'+str(last_month_nums)+'MONTH' for col in df.columns if 'STATUS' in col]
    column_dict = dict(zip(curr_column_header_list,new_column_header_list))
    #print(column_dict)
    #print(df.columns)
    df.rename(columns=column_dict,inplace=True)
    #print(df.columns)
    aggregation = dict(zip(new_column_header_list,['mean']*len(new_column_header_list)))
    #print(aggregation)
    df = df.groupby('bureau_id').agg(aggregation).reset_index()
    return df    

In [3]:
bureau = pd.read_csv('../input/bureau.zip')
bureau_bal = pd.read_csv('../input/bureau_balance.zip')

In [4]:
print(bureau.shape[0])
print(bureau_bal.shape[0])

1716428
27299925


### One-hot encoder
Perform one-hot encoder on both bureau and bureau balance datasets' categorical variables

In [5]:
bureau_bal, bureau_bal_cat_col = one_hot_encoder(bureau_bal, nan_as_category=True)
bureau, bureau_cat_col = one_hot_encoder(bureau, nan_as_category=True)

In [6]:
print(bureau_bal_cat_col)
print(bureau_cat_col)

['STATUS_0', 'STATUS_1', 'STATUS_2', 'STATUS_3', 'STATUS_4', 'STATUS_5', 'STATUS_C', 'STATUS_X', 'STATUS_nan']
['CREDIT_ACTIVE_Active', 'CREDIT_ACTIVE_Bad debt', 'CREDIT_ACTIVE_Closed', 'CREDIT_ACTIVE_Sold', 'CREDIT_ACTIVE_nan', 'CREDIT_CURRENCY_currency 1', 'CREDIT_CURRENCY_currency 2', 'CREDIT_CURRENCY_currency 3', 'CREDIT_CURRENCY_currency 4', 'CREDIT_CURRENCY_nan', 'CREDIT_TYPE_Another type of loan', 'CREDIT_TYPE_Car loan', 'CREDIT_TYPE_Cash loan (non-earmarked)', 'CREDIT_TYPE_Consumer credit', 'CREDIT_TYPE_Credit card', 'CREDIT_TYPE_Interbank credit', 'CREDIT_TYPE_Loan for business development', 'CREDIT_TYPE_Loan for purchase of shares (margin lending)', 'CREDIT_TYPE_Loan for the purchase of equipment', 'CREDIT_TYPE_Loan for working capital replenishment', 'CREDIT_TYPE_Microloan', 'CREDIT_TYPE_Mobile operator loan', 'CREDIT_TYPE_Mortgage', 'CREDIT_TYPE_Real estate loan', 'CREDIT_TYPE_Unknown type of loan', 'CREDIT_TYPE_nan']


### Outlier treatment - bureau
In bureau dataset, with columns that contains 'DAYS', we remove convert all data points have days less than -40000 to na

In [7]:
remove_date_outlier(bureau)

,bureau_id,curr_app_id,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,...,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan,CREDIT_TYPE_nan
0,8319600,528392,-2918.0,0,-2613.0,-2639.0,NaN,0,29443.50,0.00,...,0,0,0,0,0,0,0,0,0,0
1,8697773,528392,-1015.0,0,-831.0,-891.0,2223.855,0,13810.50,0.00,...,0,0,0,0,0,0,0,0,0,0
2,7742836,528392,-149.0,0,-26.0,-26.0,0.000,0,13455.00,0.00,...,0,0,0,0,0,0,0,0,0,0
3,7114563,528392,-135.0,0,230.0,-42.0,0.000,0,37350.00,0.00,...,0,0,0,0,0,0,0,0,0,0
4,7236686,528392,-47.0,0,320.0,NaN,0.000,0,315127.62,315127.62,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,8631654,962042,-2385.0,0,-1645.0,NaN,NaN,0,135000.00,NaN,...,0,0,0,0,0,0,0,0,0,0
1716424,7170241,962042,-700.0,0,-335.0,-335.0,NaN,0,67500.00,NaN,...,0,0,0,0,0,0,0,0,0,0
1716425,8842360,962042,-301.0,0,1534.0,NaN,NaN,0,67500.00,19071.00,...,0,0,0,0,0,0,0,0,0,0
1716426,8931699,962042,-1395.0,0,-299.0,-1131.0,0.000,0,225000.00,0.00,...,0,0,0,0,0,0,0,0,0,0


### Feature Enginering on Bureau Balance

1. MONTHS_BALANCE: add min, max, count of each bureau_id
2. STATUS columns: calculate recent 1, 6, 12, 18 months' mean value of each STATUS(0,1,2,3,4,5,C,X,nan)

In [8]:
agg_dict =  {'MONTHS_BALANCE': ['min', 'max', 'count']}
bureau_bal_months_balance = bureau_bal.groupby('bureau_id').agg(agg_dict).reset_index(level=[0])
bureau_bal_months_balance.columns = ['_'.join(col) for col in bureau_bal_months_balance.columns.values]
bureau_bal_months_balance.rename(columns = {'bureau_id_':'bureau_id'},inplace=True)
bureau_bal_months_balance.columns

Index(['bureau_id', 'MONTHS_BALANCE_min', 'MONTHS_BALANCE_max',
       'MONTHS_BALANCE_count'],
      dtype='object')

In [9]:
bureau_bal_last_1_months = process_bureau_balancing_with_period(bureau_bal,1)
#bureau_bal_last_1_months.head()
bureau_bal_last_6_months = process_bureau_balancing_with_period(bureau_bal,6)
#bureau_bal_last_6_months.head()
bureau_bal_last_12_months = process_bureau_balancing_with_period(bureau_bal,12)
#bureau_bal_last_12_months.head()
bureau_bal_last_18_months = process_bureau_balancing_with_period(bureau_bal,18)
#bureau_bal_last_18_months.head()

In [10]:
bureau_bal_combined = bureau_bal_last_1_months.merge(bureau_bal_last_1_months,how='left')
bureau_bal_combined = bureau_bal_combined.merge(bureau_bal_last_6_months,how='left')
bureau_bal_combined = bureau_bal_combined.merge(bureau_bal_last_12_months,how='left')
bureau_bal_combined = bureau_bal_combined.merge(bureau_bal_last_18_months,how='left')
bureau_bal_combined = bureau_bal_combined.merge(bureau_bal_months_balance,how='left')
#bureau_bal_combined.head()


In [11]:
print(bureau_bal_combined.columns.values)
print(bureau_bal_combined.shape[0])

['bureau_id' 'STATUS_0_1MONTH' 'STATUS_1_1MONTH' 'STATUS_2_1MONTH'
 'STATUS_3_1MONTH' 'STATUS_4_1MONTH' 'STATUS_5_1MONTH' 'STATUS_C_1MONTH'
 'STATUS_X_1MONTH' 'STATUS_nan_1MONTH' 'STATUS_0_6MONTH' 'STATUS_1_6MONTH'
 'STATUS_2_6MONTH' 'STATUS_3_6MONTH' 'STATUS_4_6MONTH' 'STATUS_5_6MONTH'
 'STATUS_C_6MONTH' 'STATUS_X_6MONTH' 'STATUS_nan_6MONTH'
 'STATUS_0_12MONTH' 'STATUS_1_12MONTH' 'STATUS_2_12MONTH'
 'STATUS_3_12MONTH' 'STATUS_4_12MONTH' 'STATUS_5_12MONTH'
 'STATUS_C_12MONTH' 'STATUS_X_12MONTH' 'STATUS_nan_12MONTH'
 'STATUS_0_18MONTH' 'STATUS_1_18MONTH' 'STATUS_2_18MONTH'
 'STATUS_3_18MONTH' 'STATUS_4_18MONTH' 'STATUS_5_18MONTH'
 'STATUS_C_18MONTH' 'STATUS_X_18MONTH' 'STATUS_nan_18MONTH'
 'MONTHS_BALANCE_min' 'MONTHS_BALANCE_max' 'MONTHS_BALANCE_count']
610965


### Merge processed Bureau Balance with Bureau

In [12]:
bureau = bureau.merge(right=bureau_bal_combined, right_on='bureau_id', left_on='bureau_id')

In [13]:
bureau.info(verbose=True,memory_usage=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 588522 entries, 0 to 588521
Data columns (total 79 columns):
 #   Column                                                    Non-Null Count   Dtype  
---  ------                                                    --------------   -----  
 0   bureau_id                                                 588522 non-null  int64  
 1   curr_app_id                                               588522 non-null  int64  
 2   DAYS_CREDIT                                               588522 non-null  float64
 3   CREDIT_DAY_OVERDUE                                        588522 non-null  int64  
 4   DAYS_CREDIT_ENDDATE                                       546893 non-null  float64
 5   DAYS_ENDDATE_FACT                                         347402 non-null  float64
 6   AMT_CREDIT_MAX_OVERDUE                                    181587 non-null  float64
 7   CNT_CREDIT_PROLONG                                        588522 non-null  int64  
 8   AMT_

### Feature Enginering on Bureau
create addtional features based on different aggregation types on numerical/categorical columns

In [14]:
categorical_columns = [col for col in bureau.columns if bureau[col].dtype == 'uint8']
numerical_columns = [col for col in bureau.columns if bureau[col].dtype != 'uint8']
num_aggregations = {
        'MONTHS_BALANCE_min': ['min'],
        'MONTHS_BALANCE_max': ['max'],
        'MONTHS_BALANCE_count': ['mean', 'sum'],
        'DAYS_CREDIT': ['mean','min','max'],
        'CREDIT_DAY_OVERDUE': ['mean'],
        'DAYS_CREDIT_ENDDATE': [ 'mean','min','max'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'DAYS_ENDDATE_FACT':['mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': [ 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': [ 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum']
    }
categorical_aggregations = {}
for col in categorical_columns: 
    categorical_aggregations[col] = ['mean']
aggregations = {**categorical_aggregations,**num_aggregations}

In [15]:
bureau_agg = bureau.groupby('curr_app_id').agg(aggregations)

In [16]:
bureau_agg.head()
bureau_agg.columns = ['BURO_' + col[0] + "_" + col[1].upper() for col in bureau_agg.columns.tolist()]
bureau_agg.columns

Index(['BURO_CREDIT_ACTIVE_Active_MEAN', 'BURO_CREDIT_ACTIVE_Bad debt_MEAN',
       'BURO_CREDIT_ACTIVE_Closed_MEAN', 'BURO_CREDIT_ACTIVE_Sold_MEAN',
       'BURO_CREDIT_ACTIVE_nan_MEAN', 'BURO_CREDIT_CURRENCY_currency 1_MEAN',
       'BURO_CREDIT_CURRENCY_currency 2_MEAN',
       'BURO_CREDIT_CURRENCY_currency 3_MEAN',
       'BURO_CREDIT_CURRENCY_currency 4_MEAN', 'BURO_CREDIT_CURRENCY_nan_MEAN',
       'BURO_CREDIT_TYPE_Another type of loan_MEAN',
       'BURO_CREDIT_TYPE_Car loan_MEAN',
       'BURO_CREDIT_TYPE_Cash loan (non-earmarked)_MEAN',
       'BURO_CREDIT_TYPE_Consumer credit_MEAN',
       'BURO_CREDIT_TYPE_Credit card_MEAN',
       'BURO_CREDIT_TYPE_Interbank credit_MEAN',
       'BURO_CREDIT_TYPE_Loan for business development_MEAN',
       'BURO_CREDIT_TYPE_Loan for purchase of shares (margin lending)_MEAN',
       'BURO_CREDIT_TYPE_Loan for the purchase of equipment_MEAN',
       'BURO_CREDIT_TYPE_Loan for working capital replenishment_MEAN',
       'BURO_CREDIT_TYPE_Mic

### How does client perform on active bureau credits?

In [17]:
active_bureau = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
active_bureau_agg = active_bureau.groupby('curr_app_id').agg(num_aggregations)
active_bureau_agg.columns = ['BURO_ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_bureau_agg.columns.tolist()]
print(active_bureau_agg.columns)

Index(['BURO_ACTIVE_MONTHS_BALANCE_min_MIN',
       'BURO_ACTIVE_MONTHS_BALANCE_max_MAX',
       'BURO_ACTIVE_MONTHS_BALANCE_count_MEAN',
       'BURO_ACTIVE_MONTHS_BALANCE_count_SUM', 'BURO_ACTIVE_DAYS_CREDIT_MEAN',
       'BURO_ACTIVE_DAYS_CREDIT_MIN', 'BURO_ACTIVE_DAYS_CREDIT_MAX',
       'BURO_ACTIVE_CREDIT_DAY_OVERDUE_MEAN',
       'BURO_ACTIVE_DAYS_CREDIT_ENDDATE_MEAN',
       'BURO_ACTIVE_DAYS_CREDIT_ENDDATE_MIN',
       'BURO_ACTIVE_DAYS_CREDIT_ENDDATE_MAX',
       'BURO_ACTIVE_DAYS_CREDIT_UPDATE_MEAN',
       'BURO_ACTIVE_DAYS_ENDDATE_FACT_MEAN',
       'BURO_ACTIVE_AMT_CREDIT_MAX_OVERDUE_MEAN',
       'BURO_ACTIVE_AMT_CREDIT_SUM_MEAN', 'BURO_ACTIVE_AMT_CREDIT_SUM_SUM',
       'BURO_ACTIVE_AMT_CREDIT_SUM_DEBT_MEAN',
       'BURO_ACTIVE_AMT_CREDIT_SUM_DEBT_SUM',
       'BURO_ACTIVE_AMT_CREDIT_SUM_OVERDUE_MEAN',
       'BURO_ACTIVE_AMT_CREDIT_SUM_LIMIT_MEAN',
       'BURO_ACTIVE_AMT_CREDIT_SUM_LIMIT_SUM', 'BURO_ACTIVE_AMT_ANNUITY_MAX',
       'BURO_ACTIVE_AMT_ANNUITY_MEAN', 'BUR

### How does client perform on closed/previous bureau credits?

In [18]:
closed_bureau = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
closed_bureau_agg = closed_bureau.groupby('curr_app_id').agg(num_aggregations)
closed_bureau_agg.columns = ['BURO_CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_bureau_agg.columns.tolist()]
print(closed_bureau_agg.columns)

Index(['BURO_CLOSED_MONTHS_BALANCE_min_MIN',
       'BURO_CLOSED_MONTHS_BALANCE_max_MAX',
       'BURO_CLOSED_MONTHS_BALANCE_count_MEAN',
       'BURO_CLOSED_MONTHS_BALANCE_count_SUM', 'BURO_CLOSED_DAYS_CREDIT_MEAN',
       'BURO_CLOSED_DAYS_CREDIT_MIN', 'BURO_CLOSED_DAYS_CREDIT_MAX',
       'BURO_CLOSED_CREDIT_DAY_OVERDUE_MEAN',
       'BURO_CLOSED_DAYS_CREDIT_ENDDATE_MEAN',
       'BURO_CLOSED_DAYS_CREDIT_ENDDATE_MIN',
       'BURO_CLOSED_DAYS_CREDIT_ENDDATE_MAX',
       'BURO_CLOSED_DAYS_CREDIT_UPDATE_MEAN',
       'BURO_CLOSED_DAYS_ENDDATE_FACT_MEAN',
       'BURO_CLOSED_AMT_CREDIT_MAX_OVERDUE_MEAN',
       'BURO_CLOSED_AMT_CREDIT_SUM_MEAN', 'BURO_CLOSED_AMT_CREDIT_SUM_SUM',
       'BURO_CLOSED_AMT_CREDIT_SUM_DEBT_MEAN',
       'BURO_CLOSED_AMT_CREDIT_SUM_DEBT_SUM',
       'BURO_CLOSED_AMT_CREDIT_SUM_OVERDUE_MEAN',
       'BURO_CLOSED_AMT_CREDIT_SUM_LIMIT_MEAN',
       'BURO_CLOSED_AMT_CREDIT_SUM_LIMIT_SUM', 'BURO_CLOSED_AMT_ANNUITY_MAX',
       'BURO_CLOSED_AMT_ANNUITY_MEAN', 'BUR

In [19]:
bureau_agg = bureau_agg.join(active_bureau_agg, how='left', on='curr_app_id')
bureau_agg = bureau_agg.join(closed_bureau_agg, how='left', on='curr_app_id')

In [20]:
bureau_agg=bureau_agg.reset_index()
downcast_dtypes(bureau_agg)

,curr_app_id,BURO_CREDIT_ACTIVE_Active_MEAN,BURO_CREDIT_ACTIVE_Bad debt_MEAN,BURO_CREDIT_ACTIVE_Closed_MEAN,BURO_CREDIT_ACTIVE_Sold_MEAN,BURO_CREDIT_ACTIVE_nan_MEAN,BURO_CREDIT_CURRENCY_currency 1_MEAN,BURO_CREDIT_CURRENCY_currency 2_MEAN,BURO_CREDIT_CURRENCY_currency 3_MEAN,BURO_CREDIT_CURRENCY_currency 4_MEAN,...,BURO_CLOSED_AMT_CREDIT_SUM_MEAN,BURO_CLOSED_AMT_CREDIT_SUM_SUM,BURO_CLOSED_AMT_CREDIT_SUM_DEBT_MEAN,BURO_CLOSED_AMT_CREDIT_SUM_DEBT_SUM,BURO_CLOSED_AMT_CREDIT_SUM_OVERDUE_MEAN,BURO_CLOSED_AMT_CREDIT_SUM_LIMIT_MEAN,BURO_CLOSED_AMT_CREDIT_SUM_LIMIT_SUM,BURO_CLOSED_AMT_ANNUITY_MAX,BURO_CLOSED_AMT_ANNUITY_MEAN,BURO_CLOSED_CNT_CREDIT_PROLONG_SUM
0,500000,0.000000,0.0,1.000000,0.0,0,1.0,0.0,0.0,0.0,...,18346.500,18346.50,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0
1,500002,0.666667,0.0,0.333333,0.0,0,1.0,0.0,0.0,0.0,...,98496.000,196992.00,0.0,0.0,0.0,0.0,0.0,25411.5,25411.50,0.0
2,500006,0.000000,0.0,1.000000,0.0,0,1.0,0.0,0.0,0.0,...,1125000.000,1125000.00,NaN,0.0,0.0,NaN,0.0,NaN,NaN,0.0
3,500008,0.500000,0.0,0.500000,0.0,0,1.0,0.0,0.0,0.0,...,87572.250,175144.50,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
4,500012,0.000000,0.0,1.000000,0.0,0,1.0,0.0,0.0,0.0,...,56736.000,113472.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129824,999968,0.500000,0.0,0.500000,0.0,0,1.0,0.0,0.0,0.0,...,184000.500,184000.50,0.0,0.0,0.0,NaN,0.0,0.0,0.00,0.0
129825,999972,0.333333,0.0,0.666667,0.0,0,1.0,0.0,0.0,0.0,...,25345.125,50690.25,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0
129826,999977,0.600000,0.0,0.400000,0.0,0,1.0,0.0,0.0,0.0,...,35363.250,70726.50,0.0,0.0,0.0,0.0,0.0,4108.5,2054.25,0.0
129827,999990,0.500000,0.0,0.500000,0.0,0,1.0,0.0,0.0,0.0,...,630000.000,630000.00,NaN,0.0,0.0,NaN,0.0,4014.0,4014.00,0.0


### Export processed bureau history file to staging folder

In [21]:
bureau_agg.to_csv('../staging/bureau_history.csv', index=False, compression='zip')

In [22]:
bureau_agg.columns.values

array(['curr_app_id', 'BURO_CREDIT_ACTIVE_Active_MEAN',
       'BURO_CREDIT_ACTIVE_Bad debt_MEAN',
       'BURO_CREDIT_ACTIVE_Closed_MEAN', 'BURO_CREDIT_ACTIVE_Sold_MEAN',
       'BURO_CREDIT_ACTIVE_nan_MEAN',
       'BURO_CREDIT_CURRENCY_currency 1_MEAN',
       'BURO_CREDIT_CURRENCY_currency 2_MEAN',
       'BURO_CREDIT_CURRENCY_currency 3_MEAN',
       'BURO_CREDIT_CURRENCY_currency 4_MEAN',
       'BURO_CREDIT_CURRENCY_nan_MEAN',
       'BURO_CREDIT_TYPE_Another type of loan_MEAN',
       'BURO_CREDIT_TYPE_Car loan_MEAN',
       'BURO_CREDIT_TYPE_Cash loan (non-earmarked)_MEAN',
       'BURO_CREDIT_TYPE_Consumer credit_MEAN',
       'BURO_CREDIT_TYPE_Credit card_MEAN',
       'BURO_CREDIT_TYPE_Interbank credit_MEAN',
       'BURO_CREDIT_TYPE_Loan for business development_MEAN',
       'BURO_CREDIT_TYPE_Loan for purchase of shares (margin lending)_MEAN',
       'BURO_CREDIT_TYPE_Loan for the purchase of equipment_MEAN',
       'BURO_CREDIT_TYPE_Loan for working capital replenishment